# Initialize PyImageJ

In [1]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [2]:
Fiji_Local = r"C:\Users\confocal_microscope\Desktop\Tools\Fiji.app"

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.53t

ImageJ2 2.9.0/1.54b; Java 1.8.0_362 [amd64]; 166MB of 14542MB


In [3]:
def dump_info(image, CLI_print_title:str=None):
    """A handy function to print details of an image object."""
    name = image.name if hasattr(image, 'name') else None # xarray
    if name is None and hasattr(image, 'getName'): name = image.getName() # Dataset
    if name is None and hasattr(image, 'getTitle'): name = image.getTitle() # ImagePlus
    
    if CLI_print_title is not None: print(f"--> {CLI_print_title}:\n")
    print(f"    name  : {name or 'N/A'}")
    print(f"    type  : {type(image)}")
    print(f"    dtype : {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
    print(f"    shape : {image.shape if hasattr(image, 'shape') else 'N/A'}")
    print(f"    dims  : {image.dims if hasattr(image, 'dims') else 'N/A'}\n")

Get ```seriesCount``` ( Method 1 )

In [4]:
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Ext = loci.plugins.macro.LociFunctions()

# LociFunctions = sj.jimport("loci.plugins.macro.LociFunctions")
# Ext = LociFunctions()

test_lif = r"C:\Users\confocal_microscope\Desktop\PyImageJ-test-data\20220610_CE001_palmskin_8dpf.lif"

Ext.setId(test_lif)

seriesCount = jpype.java.lang.Double[1]
print(f'seriesCount {type(seriesCount)}: {seriesCount}')
Ext.getSeriesCount(seriesCount)
print(f'seriesCount {type(seriesCount)}: {seriesCount}')

seriesCount = seriesCount[0]
print(f'seriesCount {type(seriesCount)}: {seriesCount}')
Ext.close()

seriesCount <java class 'java.lang.Double[]'>: [None]
seriesCount <java class 'java.lang.Double[]'>: [6.0]
seriesCount <java class 'java.lang.Double'>: 6.0


Get ```seriesCount``` ( Method 2 )

In [5]:
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# ImageReader = sj.jimport("loci.formats.ImageReader")
# Reader = ImageReader()

test_lif = r"C:\Users\confocal_microscope\Desktop\PyImageJ-test-data\20220610_CE001_palmskin_8dpf.lif"

Reader.setId(test_lif)

seriesCount = Reader.getSeriesCount()
print(f'seriesCount {type(seriesCount)}: {seriesCount}')

seriesCount <java class 'JInt'>: 6


# Start Process

In [6]:
import os
import re
import logging
from glob import glob
from tqdm.auto import tqdm

import  numpy as np

log: logging.Logger = logging.getLogger(name=r'{Test}_BF_analysis')
log.setLevel(logging.DEBUG)
formatter: logging.Formatter = logging.Formatter('| %(asctime)s | %(name)s | %(levelname)s | %(message)s')

stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
log.addHandler(stream_handler)


def create_new_dir(path:str, end="\n", display_in_CLI=True, use_tqdm=False):
    if not os.path.exists(path):
        # if the demo_folder directory is not exist then create it.
        os.makedirs(path)
        if use_tqdm: tqdm.write(f"path: '{path}' is created!{end}")
        elif display_in_CLI: print(f"path: '{path}' is created!{end}")

In [8]:
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# Important!!!
from ij.plugin.frame import RoiManager
from ij.plugin import ImageCalculator
rm = RoiManager()
imageCalculator = ImageCalculator()


# Create sub folder
BF_raw_lif_base = r"C:\Users\confocal_microscope\Desktop\{SRY_Lite}_SA_SL_TEST" # r"C:\Users\confocal_microscope\Desktop\新增資料夾\{SRY_Lite}_SA_SL_TEST"
dir_name = BF_raw_lif_base.split(os.sep)[-1]
dir_tiff = os.path.join(BF_raw_lif_base, f"{dir_name}--TIFF")
dir_metaimg = os.path.join(BF_raw_lif_base, f"{dir_name}--MetaImage")
dir_result = os.path.join(BF_raw_lif_base, f"{dir_name}--Result")
create_new_dir(dir_tiff, display_in_CLI=False)
create_new_dir(dir_metaimg, display_in_CLI=False)
create_new_dir(dir_result, display_in_CLI=False)


# Scan Leica LIF file
lif_path_list  = glob(os.path.normpath(f"{BF_raw_lif_base}/*.lif"))
log.info(f'lif_path_list {type(lif_path_list)}: {lif_path_list}\n')


# Create log writer
log_path = os.path.join(BF_raw_lif_base, "Logs.log")
log_writer = open(log_path, mode="w")



fish_id = 0
for i, lif_path in enumerate(lif_path_list):
    
    log.info(f"Processing ... {i+1}/{len(lif_path_list)}")
    
    Reader.setId(lif_path)
    log.info(f'LIF_FILE : {lif_path}')
    
    seriesCount = Reader.getSeriesCount()
    log.info(f'seriesCount : {seriesCount}')
    
    # Write Log
    log_writer.write("\n\n\n")
    log_writer.write(f"|-----------------------------------------  Processing ... {i+1}/{len(lif_path_list)}  ----------------------------------------- \n")
    log_writer.write(f"| \n")
    log_writer.write(f"|         LIF_FILE : {lif_path.split(os.sep)[-1]} \n")
    log_writer.write(f"| \n")


    for j in range(seriesCount):
        
        ij.IJ.run("Bio-Formats Importer", f"open={lif_path} color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{j+1}")
        img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        # dump_info(img)
        # img.show()
        img.hide()
        
        
        # Get names and image dimensions
        file_name = lif_path.split(os.sep)[-1].split(".")[0]
        image_name = img.getProp("Image name")
        seN = f"{file_name} - {image_name}"
        img_dimensions = img.getDimensions()
        log.info(f"series {j+1:{len(str(seriesCount))}}/{seriesCount} : '{seN}' , Dimensions : {img_dimensions} ( width, height, channels, slices, frames )")
        # Write Log
        log_writer.write(f"|-- processing ...  series {j+1:{len(str(seriesCount))}}/{seriesCount} in {i+1}/{len(lif_path_list)} \n")
        log_writer.write(f"|         {seN} \n")
        log_writer.write(f"|         Dimensions : {img_dimensions} ( width, height, channels, slices, frames ) \n")
        
        
        # Get fish_id and create subfolder in "--MetaImage" by fish_id
        seN_split = re.split("[fish _-]", seN)
        while "" in seN_split: seN_split.remove("")
        if (fish_id == seN_split[9]):
            log.info("      #### WARNING : Detect ' fish_id ' is same as previous processed fish --> skip this image"); # WARNING:
            # Write Log
            log_writer.write("| #### WARNING : Detect ' fish_id ' is same as previous processed fish --> skip this image \n") # WARNING:
            log_writer.write("| \n")
            continue
        else:
            fish_id = seN_split[9]
            # log.info(fish_id)
        metaimg_subfolder = os.path.join(dir_metaimg, f"Fish_{fish_id}")
        create_new_dir(metaimg_subfolder, display_in_CLI=False)
        
        
        
        slices = img_dimensions[3]
        if slices > 0:
            
            # Find focused slices
        
            #   Pick up focused slice if slices > 1
            #   Plugin ref : https://sites.google.com/site/qingzongtseng/find-focus
            #   Algorithm  : autofocus algorithm "Normalized variance"  (Groen et al., 1985; Yeo et al., 1993).
            if slices > 1:
                log.info("      #### WARNING : Number of Slices > 1, run ' Find focused slices ' ") # WARNING:
                # Write Log
                log_writer.write("| #### WARNING : Number of Slices > 1, run ' Find focused slices ' \n") # WARNING:
                ij.IJ.run(img, "Find focused slices", "select=100 variance=0.000 select_only")
                img = ij.WindowManager.getCurrentImage()
                # img.show()
                img.hide()
            # dump_info(img)
            ij.IJ.saveAsTiff(img, os.path.normpath(f"{dir_tiff}/{seN}.tif"))
            # ij.IJ.saveAs(img, "Tiff", os.path.normpath(f"{dir_tiff}/{seN}.tif"))
            # img.show()
            img.hide()


            # Convert to 8-bit and Set Scale

            ij.IJ.run("Conversions...", "scale weighted")
            ij.IJ.run(img, "8-bit", "")
            # img.show()
            img.hide()
            
            # Consociate the unit ( relationship between micron and pixel )
            #   To prevent some series of images are in different scales, for example, "20220708_CE009_palmskin_8dpf.lif"
			#   Microscope Metadata : 1 pixel = 0.0000065 m = 6.5 micron
            ij.IJ.run(img, "Set Scale...", "distance=0.3076923076923077 known=1 unit=micron")
            # img.show()
            img.hide()

        
            # Cropping image
        
            img.setRoi(50, 700, 1950, 700)
            crop_img = img.crop()
            ij.IJ.saveAsTiff(crop_img, os.path.normpath(f"{metaimg_subfolder}/{seN}_Cropped.tif"))
            # crop_img.show()
            crop_img.hide()


            # Threshold
            
            crop_img_threshold = crop_img.duplicate()
            ij.IJ.run(crop_img_threshold, "Auto Threshold", "method=Triangle white")
            ij.prefs.blackBackground = True
            ij.IJ.run(crop_img_threshold, "Convert to Mask", "")
            ij.IJ.saveAsTiff(crop_img_threshold, os.path.normpath(f"{metaimg_subfolder}/{seN}_Threshold.tif"))
            # crop_img_threshold.show()
            crop_img_threshold.hide()
            
            
            # SL and SA measurement
            
            ij.IJ.run("Set Measurements...", "area feret's display redirect=None decimal=2")
            ij.IJ.run(crop_img_threshold, "Analyze Particles...", "size=800000-4000000 show=Masks display include add")
            mask_img = ij.WindowManager.getCurrentImage()
            ij.IJ.run(mask_img, "Convert to Mask", "")
            ij.IJ.saveAsTiff(mask_img, os.path.normpath(f"{metaimg_subfolder}/{seN}--Mask.tif"))
            # mask_img.show()
            mask_img.hide()
            
            rm.runCommand("Show All with labels")
            rm_size = int(rm.getCount())
            if rm_size == 1:  # success to get fish
                mix_img = imageCalculator.run(crop_img, mask_img, "AND create")
                ij.IJ.saveAsTiff(mix_img, os.path.normpath(f"{metaimg_subfolder}/{seN}--MIX.tif"))
                # mix_img.show()
                mix_img.hide()
                
                rm.save(os.path.normpath(f"{metaimg_subfolder}/{seN}_RoiSet.zip"))
                ij.IJ.saveAs("Results", os.path.normpath(f"{dir_result}/{seN}_AutoAnalysis.csv"))
                rm.runCommand("Deselect")
                rm.runCommand("Delete") # delete ROI, otherwise, it increases infinitely
            else:
                log.info(f'      ROI in RoiManager: {rm_size}')
                log.info(f'      #### ERROR : Number of ROI not = 1')
                # Write Log
                log_writer.write(f"|         number of ROI = {rm_size} \n")
                log_writer.write("| #### ERROR : Number of ROI not = 1 \n") # ERROR:
                
                if (rm_size > 1): # delete ROI, otherwise, it increases infinitely
                    rm.runCommand("Deselect")
                    rm.runCommand("Delete")
        
        else: log.info("      #### WARNING : Can't find any Slices in this image --> skip this image") # WARNING:


        log_writer.write(f"| \n") # make Log file looks better.
        ij.IJ.run("Clear Results", "")
        ij.IJ.run("Close All", "")
    
    
    log.info("\n") # make CLI output looks better.


log_writer.close()
ij.WindowManager.closeAllWindows()

log.info(" -- finished --")

| 2023-02-13 20:53:59,257 | {Test}_BF_analysis | INFO | lif_path_list <class 'list'>: ['C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20220617_CE002_palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20220621_CE003_palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20220708_CE009_palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20220711_CE009_palmskin_11dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20220725_CE012_palmskin_7dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20220816_AI001_palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20220818_AI001_palmskin_10dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20221125_AI005_palmskin_10dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{SRY_Lite}_SA_SL_TEST\\20221127_AI005_palmskin_12dpf.lif']

| 2023-0

# Other Useful Methods 

```
ij.WindowManager.closeAllWindows()

ij.WindowManager.getCurrentImage()

img.getProperties()

ij.WindowManager.getActiveWindow()
```